imports
-

In [1]:
import sys, os
sys.version

'3.9.16 (main, Mar  8 2023, 14:00:05) \n[GCC 11.2.0]'

In [2]:
#math
import numpy as np
#capture
import cv2
import wave
#utils
from tqdm import tqdm
from IPython.display import Image
from IPython.display import clear_output
import datetime
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import time
import humanize

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
#neural
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
print(tf.__version__)

2024-05-07 01:10:15.079547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 01:10:16.028975: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.12.0


system info
-

In [5]:
for i in device_lib.list_local_devices():
    print(f'{i.device_type}|{i.name} memory limit: {humanize.naturalsize(i.memory_limit)}')
    print(i.physical_device_desc)

CPU|/device:CPU:0 memory limit: 268.4 MB

GPU|/device:GPU:0 memory limit: 11.3 GB
device: 0, name: Tesla M40, pci bus id: 0000:03:00.0, compute capability: 5.2


2024-05-07 01:10:17.156149: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /device:GPU:0 with 10821 MB memory:  -> device: 0, name: Tesla M40, pci bus id: 0000:03:00.0, compute capability: 5.2


import data
-

In [6]:
def reshape(frame):
    global WIDTH, HEIGHT
    newframe = np.zeros(WIDTH*HEIGHT).reshape(HEIGHT, WIDTH)
    if frame.shape[0] == HEIGHT and frame.shape[1] == WIDTH:
        return frame
    elif frame.shape[0] <= HEIGHT and frame.shape[1] <= WIDTH:
        newframe[:frame.shape[0],:frame.shape[1]] = frame
    elif frame.shape[0] <= HEIGHT:
        newframe[:frame.shape[0],:] = frame[:, :WIDTH]
    elif frame.shape[1] <= WIDTH:
        newframe[:,:frame.shape[1]] = frame[:HEIGHT, :]
    else:
        newframe = frame[:HEIGHT, :WIDTH]
    return newframe

def deinterlase(frame):
    even_fields=frame[::2,:];
    odd_fields=frame[1::2,:];
    return even_fields, odd_fields

In [7]:
WIDTH = int(720)
HEIGHT = int(625-1)

def normilizeFrame(frame):
    global WIDTH, HEIGHT
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #change dimentions(choose 1 of 2 options)
    #frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    frame = reshape(frame)
    
    return deinterlase(frame)
    #return frame / 255.0

def retrieve():
    global WIDTH, HEIGHT
    if cap.grab():
        ret, frame = cap.retrieve()
        if not ret:
            print('error reading frame')
            return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))
        return True, normilizeFrame(frame)
    else:
        return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))

def getNext():
    ret, (topFrame, bottomFrame) = retrieve()
    topFrame = topFrame[8:302,26:461]
    bottomFrame = bottomFrame[8:302,26:461]
    frm = np.append(topFrame, bottomFrame)
    return ret, frm.reshape(2, topFrame.shape[0], topFrame.shape[1])

def openCap(file = './dataset/video.avi'):
    global cap, half, first
    if not os.path.exists(file):
        print("No file to open!")
        exit()
    first = True
    cap = cv2.VideoCapture(file)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
openCap()

try to generate pcm frame(just for understanding)

In [8]:
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(frameCount)

#tech calc

#audio in pcm have 44100 samples in second per channel, respectively we have 88200 samples
#per frame we have 44100/25(pal) = 1764 samples(for single channel)(and we take half of frame for learning /2)

samplesPerFrame = int(44100/cap.get(cv2.CAP_PROP_FPS)/2)
print(samplesPerFrame)

1501
882


create dataset

In [9]:
dir = "./dataset/"

In [10]:
def initQueues():
    global bufR0, bufL1, bufR1, bufR2, bufL2
    bufR0 = deque()
    bufL1 = deque()
    bufR1 = deque()
    bufL2 = deque()
    bufR2 = deque()
    
    for i in range(1*16):
        bufR0.append(0)
    
    for i in range(2*16):
        bufL1.append(0)
    
    for i in range(3*16):
        bufR1.append(0)
    
    for i in range(4*16):
        bufL2.append(0)
    
    for i in range(5*16):
        bufR2.append(0)    
        
def pairwise(iterable):
    a = iter(iterable)
    if len(iterable) % 6 != 0:
        print(f'input length detected that is not divisible by 6! all data after the {len(iterable)-(len(iterable) % 6)}({len(iterable) % 6} cells) position will be ignored.')
    return zip(a, a, a, a, a, a)
    
def Abreacker(sample):
    global bufR0, bufL1, bufR1, bufR2, bufL2
    filtered = np.array([], dtype=np.uint16)
    line = np.zeros(6, dtype=np.uint16)

    for elem in pairwise(sample):
        bufR0.append(elem[1])
        bufL1.append(elem[2])
        bufR1.append(elem[3])
        bufL2.append(elem[4])
        bufR2.append(elem[5])
        
        line[0] = elem[0]
        line[1] = bufR0.popleft()
        line[2] = bufL1.popleft()
        line[3] = bufR1.popleft()
        line[4] = bufL2.popleft()
        line[5] = bufR2.popleft()
                
        filtered = np.append(filtered, line)
    
    return filtered

In [11]:
def initQueuesForDebreacker():
    global bufL0D, bufR0D, bufL1D, bufR1D, bufL2D
    bufL0D = deque()
    bufR0D = deque()
    bufL1D = deque()
    bufR1D = deque()
    bufL2D = deque()
    
    for i in range(5*16):
        bufL0D.append(0)
        
    for i in range(4*16):
        bufR0D.append(0)
    
    for i in range(3*16):
        bufL1D.append(0)
    
    for i in range(2*16):
        bufR1D.append(0)
    
    for i in range(1*16):
        bufL2D.append(0)
        
def Adebreacker(sample):
    global bufL0D, bufR0D, bufL1D, bufR1D, bufL2D
    filtered = np.array([], dtype=np.uint16)
    line = np.zeros(6, dtype=np.uint16)
    
    for elem in pairwise(sample):
        bufL0D.append(elem[0])
        bufR0D.append(elem[1])
        bufL1D.append(elem[2])
        bufR1D.append(elem[3])
        bufL2D.append(elem[4])
        
        line[0] = bufL0D.popleft()
        line[1] = bufR0D.popleft()
        line[2] = bufL1D.popleft()
        line[3] = bufR1D.popleft()
        line[4] = bufL2D.popleft()
        line[5] = elem[5]
                
        filtered = np.append(filtered, line)
    
    return filtered


In [12]:
mainAudioFile = wave.open(f'{dir}audio.wav', mode='r')

def gen():
    global mainAudioFile, samplesPerFrame
    
    #audio
    initQueues()
    mainAudioFile.rewind()
    
    if not (mainAudioFile.getparams()[0] == 2 and mainAudioFile.getparams()[2] == 44100):
        print('wrong audiofile params')
        sys.exit(1)
    
    
    #video
    openCap()
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT) -1
    
    
    while int(cap.get(cv2.CAP_PROP_POS_FRAMES)) <= frames:
        #audio
        frame = mainAudioFile.readframes(samplesPerFrame)
        
        if len(frame) < samplesPerFrame*2:
            frame = frame + b'\x00' * ((samplesPerFrame*4)-len(frame))
        
        #video
        vframe = getNext()[1]
        
        yield vframe[0][np.newaxis, :], (Abreacker(tf.io.decode_raw(frame, tf.uint16, little_endian=True).numpy()))
        
        #second frame
        frame = mainAudioFile.readframes(samplesPerFrame)
        
        if len(frame) < samplesPerFrame*2:
            frame = frame + b'\x00' * ((samplesPerFrame*4)-len(frame))
        yield vframe[1][np.newaxis, :], (Abreacker(tf.io.decode_raw(frame, tf.uint16, little_endian=True).numpy()))

        
dataset=tf.data.Dataset.from_generator(
    gen, 
    output_signature=(#tf.TensorSpec(shape=(1, int(HEIGHT/2), WIDTH), dtype=tf.uint8, name='image'),
                      tf.TensorSpec(shape=(None, 294, 435), dtype=tf.uint8, name='image'),
                      tf.TensorSpec(shape=(samplesPerFrame*2,), dtype=tf.uint16, name='audio'),
                      #tf.TensorSpec(shape=(1), dtype=tf.int32, name='frameNum')
                     )
)#.cache('./dataset/')

2024-05-07 01:10:17.242430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10821 MB memory:  -> device: 0, name: Tesla M40, pci bus id: 0000:03:00.0, compute capability: 5.2


In [13]:
tmp = list(dataset.take(1))[0]
print(len(tmp[1].numpy()))
print(tmp[1].numpy())
print(tmp)

1764
[22294     0     0 ... 37738 10604 48614]
(<tf.Tensor: shape=(1, 294, 435), dtype=uint8, numpy=
array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [149, 149, 149, ...,   0,   0,   0],
        ...,
        [149, 149, 149, ...,   0,   0,   0],
        [  0,   0,   0, ..., 149, 149, 149],
        [  0,   0,   0, ..., 149, 149, 149]]], dtype=uint8)>, <tf.Tensor: shape=(1764,), dtype=uint16, numpy=array([22294,     0,     0, ..., 37738, 10604, 48614], dtype=uint16)>)


2024-05-07 01:10:17.389458: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


#create model
-

In [14]:
#input = keras.Input(shape=(int(HEIGHT/2), WIDTH), name = 'image')
input = keras.Input(shape=(294, 435), name = 'image', dtype=tf.uint8)

dtype = tf.float32

In [15]:
x = tf.keras.layers.Rescaling(1./255, name='normalize', dtype=dtype)(input)
i = 1
x = tf.keras.layers.Conv1D(168, 1, activation='relu', name=f'conv1d_{i}', dtype=dtype)(x)
i +=1
x = tf.keras.layers.Conv1D(168, 1, activation='relu', name=f'conv1d_{i}', dtype=dtype)(x)

print(x.shape)

x = tf.keras.layers.Reshape((x.shape[1], x.shape[2], 1))(x)
#
i = 1
x = tf.keras.layers.Conv2D(1, (1, 4), strides=(1,4), activation='sigmoid', name=f'conv2d_{i}', dtype=dtype)(x)
#i +=1
#x = tf.keras.layers.Conv2D(1, (1, 16), strides=(1,16), activation='sigmoid', name=f'conv2d_{i}', dtype=dtype)(x)
#i +=1
#x = tf.keras.layers.Conv2D(3, 32, activation='sigmoid', name=f'conv2d_{i}')(x)
#
#x = tf.keras.layers.Flatten(name='Flatten')(x)
#x = tf.keras.layers.Dense((samplesPerFrame*2), activation='relu', use_bias=False, dtype=dtype)(x)

(None, 294, 168)


In [16]:
x = tf.keras.layers.Flatten(name='Flatten2')(x)
print(x.shape)
print(f'diff total:{x.shape[1]-(samplesPerFrame*2)}') 

(None, 12348)
diff total:10584


In [17]:
x = tf.keras.layers.Dense((samplesPerFrame*2), activation='relu', use_bias=True, dtype=dtype)(x)
x = tf.keras.layers.Rescaling(65535, name='normalize_touint')(x)

In [18]:
model = keras.Model(input, x)
model.summary()

#tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=True, show_dtype=False, show_layer_names=True, rankdir="TB", expand_nested=True, dpi=96, layer_range=None, show_layer_activations=True)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 294, 435)]        0         
                                                                 
 normalize (Rescaling)       (None, 294, 435)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 294, 168)          73248     
                                                                 
 conv1d_2 (Conv1D)           (None, 294, 168)          28392     
                                                                 
 reshape (Reshape)           (None, 294, 168, 1)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 294, 42, 1)        5         
                                                                 
 Flatten2 (Flatten)          (None, 12348)             0     

compile and learn
-

In [19]:
model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['binary_accuracy', 'mean_squared_error', 'MeanAbsolutePercentageError'])
loss = 1
prew_loss = loss
step = 0    
fails = 0
global_min = 0

In [20]:
!rm -rf ./logs/ ./model/ ./output/
!mkdir logs model output output/flac

/usr/bin/zsh: /mnt/trashbin/alex/miniconda3/envs/tf/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /mnt/trashbin/alex/miniconda3/envs/tf/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)


In [23]:
testfile = 'testfiles/pal/Portal2-3x10-Your_Precious_Moon.avi'
limit = 1    #limiting in seconds

def predict():
    global step, testfile, vframe, model, cap, times
    initQueuesForDebreacker()
    
    openCap(testfile)
    bar = tqdm(total=25*limit, ncols=100, position=0, leave=True)
    audio_buffer = np.array([], dtype=np.uint16)

    file = wave.open(f'output/{step}.wav', mode='w')
    file.setnchannels(2) 
    file.setsampwidth(2)
    file.setframerate(44100)
    ret, vframe = getNext()
    frst = True
    times = np.zeros(4*25*limit).reshape(4, 25*limit)
    i = 0
    
    
    while ret:
        aframe = Adebreacker(model.predict(vframe, verbose=0, batch_size=1)[0].astype(np.uint16))
        
        audio_buffer = np.concatenate((audio_buffer, np.frombuffer(aframe.tobytes(), dtype=np.uint16)))
        
        if len(aframe) != (samplesPerFrame*2):
            print(f'len != (samplesPerFrame*2)')
        
        if frst:
            frst = False
            #file.writeframes(aframe[480:].tobytes())
            file.writeframes(aframe.tobytes())
        else:
            file.writeframes(aframe.tobytes())
        
        bar.update(1)
        ret, vframe = getNext()
        
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) > 25*limit:
            ret = False
            
        i = i + 1
    
    bar.close()
    file.close()
    
    sample_rate = 44100
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    spectrogram_data, freqs, times, im = ax1.specgram(audio_buffer, Fs=sample_rate, cmap='inferno')
    ax1.set_title('Спектрограмма')
    ax1.set_xlabel('Время')
    ax1.set_ylabel('Частота')
    plt.colorbar(im, ax=ax1)

    time_array = np.linspace(0, len(audio_buffer) / sample_rate, len(audio_buffer))
    ax2.plot(time_array, audio_buffer)
    ax2.set_title('Аудиоданные')
    ax2.set_xlabel('Время (с)')
    ax2.set_ylabel('Амплитуда')

    plt.style.use('dark_background')

    output_path = f'output/{step}_spec_and_audio.jpg'
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    !LD_LIBRARY_PATH= /usr/bin/ffmpeg -loglevel error -hide_banner -i output/{step}.wav -y -fs 24M output/flac/{step}.flac
    #IPython.display.display(IPython.display.Audio(f'output/flac/{step}.flac'))

In [24]:
predict()

100%|███████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 14.11it/s]


/usr/bin/zsh: /mnt/trashbin/alex/miniconda3/envs/tf/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)


In [25]:
tmp = model.predict(vframe, verbose=0, batch_size=1)[0]
print(tmp)
print(len(tmp))
print(min(tmp), max(tmp))

[57801.58  19688.984     0.    ... 23089.31      0.    20922.   ]
1764
0.0 143160.08


In [26]:
epochs=5
while loss >= 0.001:    
    log_dir = f'logs/fit/stp{step+epochs}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    tf.debugging.set_log_device_placement(True)
    fitHistory = model.fit(dataset, epochs=epochs, callbacks=[tensorboard_callback])
    
    clear_output(wait=True)
    model.save(f'model/stp{step+epochs}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}', save_format="h5")
    
    step = step +epochs
    prew_loss = loss
    loss = np.min(fitHistory.history['loss'])
    
    print(f'{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")} step: {step} min loss: {loss}(from prew ehoch:{loss-prew_loss}/current: {np.min(fitHistory.history["loss"]) - np.max(fitHistory.history["loss"])}/from global min: {loss - global_min}) fails: {fails}')

    if global_min > loss or global_min == 0:
        global_min = loss
        fails = 0
    else:
        fails += 1
    
    if fails < 2:
        predict()
    
    if fails >= 5:
        #learn fail, need to rebuild model
        loss = 0

20240507-012009 step: 15 min loss: 26600.185546875(from prew ehoch:0.08984375/current: -0.220703125/from global min: 0.08984375) fails: 0


100%|███████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 14.04it/s]


/usr/bin/zsh: /mnt/trashbin/alex/miniconda3/envs/tf/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
Epoch 1/5
3002/3002 [==============================] - 32s 11ms/step - loss: 26600.1855 - binary_accuracy: 6.6477e-04 - mean_squared_error: 1027946176.0000 - mean_absolute_percentage_error: 751022848.0000
Epoch 2/5
 179/3002 [>.............................] - ETA: 28s - loss: 26604.3242 - binary_accuracy: 4.8513e-04 - mean_squared_error: 1028080192.0000 - mean_absolute_percentage_error: 9315819520.0000


KeyboardInterrupt

